In [1]:
import numpy as np 

In [2]:
def hist_dx(x, dx, dt=1):
    hist, bins = np.histogram(x, bins=np.arange(min(x), max(x) + dx, dx)) # define os bins de acorodo com dx
    dist_den = hist / (dt * dx)
    bin_centers = bins[:-1] + dx / 2  # Centros dos bins
    return hist, bin_centers, dist_den

def cond_prob(x, x0, x1, dx):
    hist_zh, bin_centers, dist_den = hist_dx(x, dx, dt=1)
    trans_count = np.zeros_like(hist_zh, dtype=float) #zeros_like cria uma matriz com zeros do tamanho da referência
    s = 2  # estado inicial definido
    tpx = []  # armazenamento dos estados
    
    # Garante que x0 < x1
    if x0 > x1:
        x0, x1 = x1, x0
    
    for val in x:
        tpx.append(val)

        if s == 2: #  garante a troca de estados de acordo com os parametros definidos
            if val <= x0:
                s = 0
            elif val >= x1:
                s = 1

        if val <= x0:
            if s == 1:  # se for menor que o x0 e estava no estado um conta a transição
                hist_vals, _ = np.histogram(tpx, bins=np.arange(min(x), max(x) + dx, dx))
                trans_count += hist_vals
            s = 0
            tpx = []  # reinicia a lista de estados 

        elif val >= x1:
            if s == 0:  # se for maior que o x1 e estava no estado zero conta a transição
                hist_vals, _ = np.histogram(tpx, bins=np.arange(min(x), max(x) + dx, dx))
                trans_count += hist_vals
            s = 1
            tpx = [] # reinicia a lista de estados

    # Calculo do p(TP|x) = transições / frequência total
    ptpx = np.divide(trans_count, hist_zh, out=np.zeros_like(hist_zh, dtype=float), where=hist_zh > 0)

    return ptpx, bin_centers, dist_den

In [3]:
x = np.genfromtxt('trajectory_file')#, dtype= None, delimiter= None)
# print(type(x))

# Parameters
x0 = 30
x1 = 37
dx = 0.5  # Ideal range: 0.1 to 0.5

In [4]:
# Calcula o p(TP|x) e o Histograma
ptpx_result, bin_centers, dist_den = cond_prob(x, x0, x1, dx)

TPX =  np.stack((bin_centers, ptpx_result), axis=-1)
HIST =  np.stack((bin_centers, dist_den), axis=-1)

np.savetxt("TPX_vitor", TPX, fmt="%5.7f")
np.savetxt("hist_vitor", HIST, fmt="%5.7f")